## Scenario 3: Multiple data scientists working on multiple ML models

MLflow setup:
* Tracking server: yes, remote server (EC2).
* Backend store: postgresql database.
* Artifacts store: s3 bucket.

The experiments can be explored by accessing the remote server.

The exampe uses AWS to host a remote server. In order to run the example you'll need an AWS account. Follow the steps described in the file `mlflow_on_aws.md` to create a new AWS account and launch the tracking server. 

In [1]:
import boto3
import mlflow
import os
from mlflow.tracking import MlflowClient

os.environ["AWS_PROFILE"] = "" # fill in with your AWS profile. More info: https://docs.aws.amazon.com/sdk-for-java/latest/developer-guide/setup.html#setup-credentials

TRACKING_SERVER_HOST = "ec2-54-159-199-211.compute-1.amazonaws.com" # fill in with the public DNS of the EC2 instance
mlflow.set_tracking_uri(f"http://{TRACKING_SERVER_HOST}:5000")

In [2]:
print(f"tracking URI: '{mlflow.get_tracking_uri()}'")

tracking URI: 'http://ec2-54-159-199-211.compute-1.amazonaws.com:5000'


In [3]:
mlflow.search_experiments()

MlflowException: API request to http://ec2-54-159-199-211.compute-1.amazonaws.com:5000/api/2.0/mlflow/experiments/search failed with exception HTTPConnectionPool(host='ec2-54-159-199-211.compute-1.amazonaws.com', port=5000): Max retries exceeded with url: /api/2.0/mlflow/experiments/search (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x12f871070>: Failed to establish a new connection: [Errno 61] Connection refused'))

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score

mlflow.set_experiment("my-experiment-1")

with mlflow.start_run():

    X, y = load_iris(return_X_y=True)

    params = {"C": 0.1, "random_state": 42}
    mlflow.log_params(params)

    lr = LogisticRegression(**params).fit(X, y)
    y_pred = lr.predict(X)
    mlflow.log_metric("accuracy", accuracy_score(y, y_pred))

    mlflow.sklearn.log_model(lr, artifact_path="models")
    print(f"default artifacts URI: '{mlflow.get_artifact_uri()}'")

default artifacts URI: 's3://mlflow-artifacts-remote-pythondance/1/d0c6854d7b854cfbbcff3c1fd393ca23/artifacts'


In [ ]:
mlflow.search_experiments()

[<Experiment: artifact_location='s3://mlflow-artifacts-remote-pythondance/1', creation_time=1718314630988, experiment_id='1', last_update_time=1718314630988, lifecycle_stage='active', name='my-experiment-1', tags={}>,
 <Experiment: artifact_location='s3://mlflow-artifacts-remote-pythondance/0', creation_time=1718313181595, experiment_id='0', last_update_time=1718313181595, lifecycle_stage='active', name='Default', tags={}>]

### Interacting with the model registry

In [ ]:
from mlflow.tracking import MlflowClient


client = MlflowClient(f"http://{TRACKING_SERVER_HOST}:5000")

In [ ]:
client.search_registered_models()

[]

In [ ]:
runs = client.search_runs(experiment_ids=['1'])
run_id = runs[0].info.run_id

mlflow.register_model(
    model_uri=f"runs:/{run_id}/models",
    name='iris-classifier'
)

Successfully registered model 'iris-classifier'.
2024/06/13 14:34:39 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: iris-classifier, version 1
Created version '1' of model 'iris-classifier'.


<ModelVersion: aliases=[], creation_timestamp=1718318079416, current_stage='None', description='', last_updated_timestamp=1718318079416, name='iris-classifier', run_id='d0c6854d7b854cfbbcff3c1fd393ca23', run_link='', source='s3://mlflow-artifacts-remote-pythondance/1/d0c6854d7b854cfbbcff3c1fd393ca23/artifacts/models', status='READY', status_message='', tags={}, user_id='', version='1'>